In [15]:
"""import graph_tool.all as gt
from hSBM_Topicmodel.sbmtm import sbmtm """

import pandas as pd
from collections import Counter
import nltk
import string
import regex as re
import numpy as np
import ast
""" from spacy.lang.da.stop_words import STOP_WORDS """

' from spacy.lang.da.stop_words import STOP_WORDS '

## Preprocessing

In [17]:
df = pd.read_csv("tweets_final.zip", compression='zip')
df.head()

/Users/jeppefoldberg/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_full_text,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,RT @madsnyvold: Nogle sukker over stormflod og...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold']
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,@COWIdk konkluderer at @kystdk's højvandsstati...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,Forskellen er at @Cowidk inkluderer historiske...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,@kristianpagh Kender du denne rapport for KBH?...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,@kristianpagh er den for høj i køge? -Kbh er j...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']


In [19]:
# removing duplicates from 
df.drop_duplicates(subset='tweet_id') 

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_full_text,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,RT @madsnyvold: Nogle sukker over stormflod og...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold']
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,@COWIdk konkluderer at @kystdk's højvandsstati...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,Forskellen er at @Cowidk inkluderer historiske...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']"
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,@kristianpagh Kender du denne rapport for KBH?...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,@kristianpagh er den for høj i køge? -Kbh er j...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340366,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Sun Mar 14 11:55:43 +0000 2021,...,Vi lancerer i dag stort udspil for plantebaser...,False,{},16.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340367,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Mon Mar 15 11:26:58 +0000 2021,...,Aftale om sommerens festivaler præsenteres om ...,False,{},7.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340368,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Tue Apr 27 20:05:33 +0000 2021,...,"Tror I, at vi er dumme? Regeringen reklamerer ...",False,{},33.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[]
340369,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Wed Apr 28 06:01:04 +0000 2021,...,"@hansersej Det er vi fuldt bevidste om, hvilke...",False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['hansersej']


In [20]:
def preprocess(text, tokenizer=nltk.tokenize.casual.TweetTokenizer()):
    '''takes in text and removes urls, mentions, numbers and punctuation'''
    if type(text) == str:
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text)
        text = re.sub(r'https?', '', text)  # removes the extra weird https left in the text. 
        text = re.sub(r'\.\.\.', '', text)  # we do not care if people use ...
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[0-9]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = tokenizer.tokenize(text)
    else: 
        return np.nan
    return text

In [21]:
df['preprocessed_text'] = df.tweet_full_text.apply(preprocess)

In [18]:
df.to_csv('all_tweets_preprocessed.zip', compression='zip')

In [27]:
df

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,user_statuses_count,user_created_at,tweet_created_at,...,tweet_retweeted,tweet_retweeted_status,tweet_retweet_count,tweet_is_quote_status,tweet_quoted_status,tweet_quote_count,tweet_entities,tweet_hashtags,tweet_mentions,preprocessed_text
0,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 08:09:20 +0000 2017,...,False,{'created_at': 'Mon Jan 09 07:56:53 +0000 2017...,2.0,False,{},NaN,"{'hashtags': [{'text': 'kystsikring', 'indices...",['kystsikring'],['madsnyvold'],"[RT, Nogle, sukker, over, stormflod, og, høj, ..."
1,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:17:52 +0000 2017,...,False,{},1.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']","[konkluderer, at, s, højvandsstatistik, undere..."
2,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:21:08 +0000 2017,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],"['COWIdk', 'kystdk']","[Forskellen, er, at, inkluderer, historiske, k..."
3,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 13:48:11 +0000 2017,...,False,{},0.0,True,{'created_at': 'Mon Jan 09 13:17:52 +0000 2017...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh'],"[Kender, du, denne, rapport, for, KBH]"
4,152954171,Aslak Grinsted,AGrinsted,NaN,I'm a climate scientist at the Niels Bohr Inst...,3216.0,1864,7636.0,Mon Jun 07 09:01:09 +0000 2010,Mon Jan 09 17:03:43 +0000 2017,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['kristianpagh'],"[er, den, for, høj, i, køge, Kbh, er, jo, dere..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340366,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Sun Mar 14 11:55:43 +0000 2021,...,False,{},16.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Vi, lancerer, i, dag, stort, udspil, for, pla..."
340367,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Mon Mar 15 11:26:58 +0000 2021,...,False,{},7.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Aftale, om, sommerens, festivaler, præsentere..."
340368,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Tue Apr 27 20:05:33 +0000 2021,...,False,{},33.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],[],"[Tror, I, at, vi, er, dumme, Regeringen, rekla..."
340369,360234529,Zenia Stampe,zeniastampe,Folketinget,Medlem af Folketinget for Radikale Venstre. Ku...,20661.0,1015,1539.0,Mon Aug 22 22:18:23 +0000 2011,Wed Apr 28 06:01:04 +0000 2021,...,False,{},0.0,False,{},NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",[],['hansersej'],"[Det, er, vi, fuldt, bevidste, om, hvilket, ku..."


In [28]:
tokens = df['preprocessed_text']
handles = df['user_name']

In [29]:
hsbm_df = pd.DataFrame({'handles': handles, 'tokens': tokens})

In [33]:
hsbm_df_new = pd.read_csv('hsbm_df.zip', compression='zip', index_col=0)

In [45]:
hsbm_df_new.loc[hsbm_df_new.tokens.str.contains('http')]

,handles,tokens
31,Aslak Grinsted,"['RT', 'colleague', 'Jørgen', 'Peder', 'Steffe..."
39,Aslak Grinsted,"['RT', 'København', 'bliver', 'formentlig', 'l..."
73,Aslak Grinsted,"['RT', 'Läntagletscher', 'glacier', 'Switzerla..."
168,Anders Stouge,"['RT', 'TVavisen', 'lagde', 'vejen', 'forbi', ..."
171,Anders Stouge,"['RT', 'HKH', 'Kronprinsen', 'uddeler', 'pris'..."
...,...,...
339856,Zenia Stampe,"['RT', 'Dalai', 'Lama', 'har', 'en', 'forsonen..."
339877,Zenia Stampe,"['RT', 'Dejlig', 'tur', 'i', 'Mjølnerparken', ..."
339938,Zenia Stampe,"['RT', 'Zenia', 'Stampe', 'om', 'krisen', 'hos..."
340020,Zenia Stampe,"['RT', 'Støjberg', 'mener', 'at', 'forholdene'..."


In [26]:
# save it to 
compression = dict(method='zip', archive_name= 'hsbm_df.csv')
hsbm_df.to_csv('hsbm_df.zip', compression=compression)

## Running the hSBM

In [ ]:
def extract_dict(string):
    """Helper function to extract dict from string if string exists 
    else the function returns an empty dict (Thanks, Tweepy...)
    Here it is used to extract lists; that works?!"""
    try:
        out = ast.literal_eval(string)
    except:
        out = dict()
    return out

hsbm_df["tokens"] = hsbm_df.tokens.apply(lambda x: extract_dict(x))

In [ ]:
handles = df['handles']
docs = df['tokens']
# stopwords imported in the beginning; we wan't to remove them! 
docs = [[word for word in doc if word not in STOP_WORDS] for doc in docs]

In [ ]:
# Remove infrequent words; Snorres solution
cutoff = 5
c = Counter()
for doc in docs:
    c.update(Counter(doc))
vocab = c.most_common(25000)
vocab = set([word for word, count in vocab if count > 5])
# remove words
docs = [[w for w in doc if w in vocab] for doc in docs]

# the main idea
Take in the tweets from all of our actors and group these into topics with the HSBM: if it finds a climate topic we would like to see what words allows us to identify these topics and try to expand our keywords list by including these words. 

In [ ]:
# step 3
model = sbmtm()

## we have to create the document network from the corpus
model.make_graph(docs)

gt.seed_rng(42) ## seed for graph-tool's random number generator --> same results
model.fit()

In [ ]:
model.save_graph(filename = 'graph_full_dataset_nostopwords.xml.gz')
model.topics

In [ ]:
def get_topic_df(topics):
  '''takes in model.topic and returns a df that can be used to save the topics
  could (and probably should) be rewritten with list comprehensions!'''
  topic_nr = []
  words = []
  weights = []
  for topic in topics: 
    for word in topics[topic]:
      topic_nr.append('topic ' + str(topic))
      words.append(word[0])
      weights.append(word[1])

  return pd.DataFrame({'topic_nr': topic_nr,
                'words': words,
                'weights': weights})

In [ ]:
topic_df = get_topic_df(topics)

In [ ]:
topic_df.to_csv('topics_6000_nostopwords.csv')
# for looking at individual topics
#topic_df.loc[topic_df.topic_nr == 'topic 8']

## Getting new and more keywords with the topics and a word2vec model

In [6]:
from gensim.models import KeyedVectors

In [7]:
path = 'w2v/f500.model.txtvectors'
w2v = KeyedVectors.load_word2vec_format('w2v/dsl_skipgram_2020_m5_f500_epoch2_w5.model.w2v.bin', binary=True)

In [10]:
w2v.most_similar(positive=[u'klimalov'])

[('klimalovgivning', 0.7582038640975952),
 ('klimaloven', 0.7433066368103027),
 ('klimatraktat', 0.6894715428352356),
 ('energilov', 0.6751753091812134),
 ('københavneraftale', 0.6731733083724976),
 ('immigrationsreform', 0.6696126461029053),
 ('energipakke', 0.6614158153533936),
 ('klimaplan', 0.6599331498146057),
 ('kyotoaftale', 0.6560425162315369),
 ('finansreform', 0.655942440032959)]